### Put the data sets into a common format

In [1]:
import pandas as pd
import anndata

c:\Users\Owner\anaconda3\envs\curate\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\Owner\anaconda3\envs\curate\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
# Specify file path and read in data
base = 'D:/GitHub/Data/NSCLC/'
ds_path = f'{base}Wang2024/GSE194070_Raw_gene_counts_matrix.txt'
ds = pd.read_csv(ds_path, sep='\t')

MemoryError: Unable to allocate 162. KiB for an array with shape (20754,) and data type int64

In [13]:
ds

,Gene,2018jz11_AAACCTGAGGGATCTG,2018jz11_AAACCTGCACATTCGA,2018jz11_AAACCTGTCAAAGACA,2018jz11_AAACGGGCATACTCTT,2018jz11_AAACGGGGTACCTACA,2018jz11_AAACGGGGTACGCACC,2018jz11_AAACGGGGTCCAGTGC,2018jz11_AAACGGGGTGTCCTCT,2018jz11_AAACGGGTCCAGAAGG,...,2018jz63_TTTGCGCTCACAAACC,2018jz63_TTTGCGCTCTGCTGTC,2018jz63_TTTGGTTAGTATGACA,2018jz63_TTTGGTTTCAATCACG,2018jz63_TTTGGTTTCCGCGGTA,2018jz63_TTTGGTTTCGTCCAGG,2018jz63_TTTGTCAAGGCTCATT,2018jz63_TTTGTCAAGTGGTAAT,2018jz63_TTTGTCAAGTTGAGTA,2018jz63_TTTGTCACAGCAGTTT
0,AL669831.5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,LINC00115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,FAM41C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NOC2L,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,KLHL17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20749,NLGN3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20750,AVPR2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20751,CCIN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20752,ACTN3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


     -------------------------------------- 387.9/387.9 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 77.9/77.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.0/2.0 MB 32.2 MB/s eta 0:00:00
     ------------------------------------- 756.0/756.0 kB 46.6 MB/s eta 0:00:00
     ------------------------------------- 289.6/289.6 kB 17.5 MB/s eta 0:00:00
     ------------------------------------- 731.3/731.3 kB 45.1 MB/s eta 0:00:00
     ------------------------------------- 869.2/869.2 kB 53.7 MB/s eta 0:00:00
     --------------------------------------- 51.9/51.9 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 351.1/351.1 kB 21.3 MB/s eta 0:00:00
     ---------------------------------------- 124.4/124.4 kB ? eta 0:00:00
  

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
